



# Algorithm Change since P2: 
- import warnings
- Import pandas module 
- Import folium module
- Import numpy module
- read dataset ‘NYC_airbnb.csv’ and 'nyc_restaurants.csv' 
- create list for NYC areas (boroughts)
- While true, and Prompt for the borough in New York City.
- if the user's input isn't in the list it will loop again and prompt user again for borough
- then it will continue and prommpt user to choose room type 
- then prompt user for prince range (0-50, 51-100,101-150,150-200,200+)
- if the user doesn't choose from those ranges, then it will continue and propmt for the price range again
- add a price group column to dataframe 
- filter out dataset based on borough and room type and print out dataframe
- use folium map to plot five of the Airbnb options (the one's that were in the dataframe). 
- prompt user to choose which airbnb they want from the list
- after they choose, that chosen one will be plotted on the open street maps
- create empty list for list of streets, restaurants, and cuisines
- for names in street names, append to list of streets, can't go over 150,000
- for names in restaurants names, append to list of restaurants, can't go over 150,000
- for names in restaurant cuisines, append to list of restaurant cuisines, can't go over 150,000
- define function for the user's street and prompt them for which street their airbnb is on
- define function to check if there are restaurants in that street
- define function to find number of restaurants in that street and start count
- define function and prompt user for cuisine type 
- if the user street and cuisine type work, print out dataframe of those restaurants 
- print out "Hope you found the right Airbnb and restaurant of choice" 
- define main function where user street equals to get user street
- if: check if restaurant is in the user's street and it's true it would print how many restaurants there are on that street
- else print there are no restaurants 

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
! pip install folium
import folium

In [5]:
# Read data from csv
airbnb_data_origin= pd.read_csv('NYC_airbnb.csv')
airbnb_data = airbnb_data_origin.filter(['neighbourhood_group', 'price', 'room_type', 'number_of_reviews', 'latitude',
                                         'longitude', 'name', 'host_id'])
restaurants = pd.read_csv('nyc_restaurants.csv')

# Filter by neighborhood
list = ["Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island"]
while True:
    user_nbhd = input('Choose from Manhattan, Brooklyn, Queens, Bronx, Staten Island or print Quit to quit: ')
    if user_nbhd == "Quit":
        break
    if user_nbhd in list:
        user_nbhd_result = airbnb_data[airbnb_data['neighbourhood_group'] == user_nbhd]
    else:
        print('Please choose from "Manhattan", "Brooklyn", "Queens", "Bronx", "Staten Island" or enter "Quit" to quit')
        continue
# Filter by room type
    list2 = ['Private room', 'Entire home/apt', 'Shared room']
    while True:
        user_room = input("Choose from 'Private room', 'Entire home/apt', 'Shared room'")
        if user_room in list2:
            user_room_result = user_nbhd_result[user_nbhd_result['room_type'] == user_room]
            break
        else:
            print('Please choose from "Private room", "Entire home/apt", "Shared room"')
            continue
    # Filter by price group
    while True:
        user_room_result['price_group'] = np.nan
        user_room_result['price_group'][user_room_result['price'] <= 50] ='0-50'
        user_room_result['price_group'][user_room_result['price'].between(51, 101)] = '51-100'
        user_room_result['price_group'][user_room_result['price'].between(101, 151)] = '101-150'
        user_room_result['price_group'][user_room_result['price'].between(151, 201)] = '151-200'
        user_room_result['price_group'][user_room_result['price'] > 200] ='200+'
        list3 = ['0-50', '51-100', '101-150','151-200','200+']
        user_price = input(f"Choose price range from '0-50', '51-100', '101-150','151-200','200+'")
        if user_price in list3:
            user_price_result = user_room_result[user_room_result['price_group'] == user_price]
            # Sort the rooms by number of reviews from high to low
            final_sort = user_price_result.sort_values(by=['number_of_reviews'], ascending=False)
            CENTER_NYC = (40.7128, -74.0060)
            map = folium.Map(location=CENTER_NYC, zoom_start=12)
            #plot the airbnbs on the map to let users make a better decision
            for row in final_sort.head(5).to_records():
                pos = (row['latitude'], row['longitude'])
                marker = folium.Marker(location=pos, popup="%s, ID: %s" % (row['name'], row['host_id'])
                                      )
                map.add_child(marker)
            display(map)
           
            final_choice = final_sort.head(5).filter(['name', 'host_id', 'price'])
            final_df = pd.DataFrame(final_choice)
            display(final_df)
           
            name_choice = input('enter the name you want to stay: ')
            name1 = airbnb_data[airbnb_data['name'] == name_choice]
            name_df = pd.DataFrame(name1)
           #plot the user's choices on the map
            map2 = folium.Map(location=CENTER_NYC, zoom_start=12)
            pos2 = (name_df['latitude'], name_df['longitude'])
            marker2 = folium.Marker(location=pos2, popup="%s" % (name_df['name']))
            map2.add_child(marker2)
            display(map2)
            display(name_df)
           
            df = pd.read_csv('nyc_restaurants.csv')
            street_names = df['Street']
            res_names = df['Name']
            res_cuisines = df['Cuisine Description']
            list_of_streets = []
            list_of_restaurants = []
            list_of_cuisines = []
            i = 0
           
            for names in street_names:
                list_of_streets.append(names)
                if (len(list_of_streets) > 150000):
                    break
            for names in res_names:
                list_of_restaurants.append(names)
                if (len(list_of_restaurants) > 150000):
                    break
            for names in res_cuisines:
                list_of_cuisines.append(names)
                if (len(list_of_cuisines) > 150000):
                    break

            def get_user_street():
                user_street = input("Which street or avenue is your AirBnB on/near? (please enter): ")
                return user_street

            def check_res_in_street(user_street):
                if (user_street) in list_of_streets:
                    return True
                else:
                    return False

            def find_num_of_res(user_street):
                count = 0
                for words in list_of_streets:
                    if (words == user_street):
                        count += 1
                return count

            def ask_show_more(user_street):
                    cuisine_type = input("Enter cuisine: ")
                    food = restaurants[ (restaurants['Street'] == user_street) & (restaurants['Cuisine Description'] == cuisine_type)]
                    value_count = {}
                    value_count['Quantity'] = food['Name'].value_counts()
                    count_df = pd.DataFrame(value_count)
                    display(count_df)
                    print("Hope you found the right AirBnB and restaurant of your choice!")

            def main():        
                user_street = get_user_street()

                if (check_res_in_street(user_street) == True):
                    num_res_in_street = find_num_of_res(user_street)
                    if (num_res_in_street > 1):
                        print("There are " + str(num_res_in_street) + " restaurants on this street.")
                        ask_show_more(user_street)
                    else:
                        print("There is 1 restaurant on this street.")
                        ask_show_more(user_street)
                else:
                    print("There are no restaurants on this street. Start over.")
                    main()
           
            main()
           
        else:
            print("Please choose from '0-50', '51-100', '101-150','151-200','200-'")
            continue
           
        break
            # break to exit this while loop

Choose from Manhattan, Brooklyn, Queens, Bronx, Staten Island or print Quit to quit: Manhattan
Choose from 'Private room', 'Entire home/apt', 'Shared room'Private room
Choose price range from '0-50', '51-100', '101-150','151-200','200+'200+


,name,host_id,price
8871,[206] 5 min WALK to Times Square,35524316,255
8854,[205] 5 min WALK to Times Square,35524316,210
2148,"Village Life, 2-Bedroom, East Village",2203885,420
14591,RENOVATED/BEST LOCATION in MIDTOWN,60688035,250
8872,[306] 5 min WALK to Times Square,35524316,245


enter the name you want to stay: Village Life, 2-Bedroom, East Village


,neighbourhood_group,price,room_type,number_of_reviews,latitude,longitude,name,host_id
2148,Manhattan,420,Private room,182,40.72399,-73.98134,"Village Life, 2-Bedroom, East Village",2203885


Which street or avenue is your AirBnB on/near? (please enter): Avenue A
There are 436 restaurants on this street.
Enter cuisine: Japanese


,Quantity
Takahachi Restaurant,36


Hope you found the right AirBnB and restaurant of your choice!
Choose from Manhattan, Brooklyn, Queens, Bronx, Staten Island or print Quit to quit: Quit
